In [1]:
# Import libraries
import numpy as np
import pandas as pd
import pickle
import os
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor as RF_RGR
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

# Import user functions
from helper_functions import get_comp_avg, get_asymmetry

SEP = os.sep

## READ RAW DATASET

In [2]:
df = pd.read_csv(f"database_raw{SEP}db_hea_raw.csv")
print(f"df shape: {df.shape}")
df.head(5)

df shape: (218, 3)


,alloy_name,phases,VHN
0,Hf0.5Nb0.5Ta0.5Ti1.5Zr,BCC,301.0
1,NbTiV2Zr,BCC,304.0
2,NbTiVZr,BCC,335.0
3,HfNbTaTiZr,BCC,340.0
4,Al0.3HfNbTaTiZr,BCC,353.0


## CREATE ALLOY FEATURES

In [3]:
alloy_name_list = df["alloy_name"]
df_feats = pd.DataFrame()

# create asymmetry features
df_feats["r_asymm"] = get_asymmetry(alloy_name_list, feat_key="r")
df_feats["EN_Pauling_asymm"] = get_asymmetry(alloy_name_list, feat_key="EN_Pauling")
df_feats["E_GPa_asymm"] = get_asymmetry(alloy_name_list, feat_key="E_GPa")
df_feats["G_GPa_asymm"] = get_asymmetry(alloy_name_list, feat_key="G_GPa")
df_feats["K_GPa_asymm"] = get_asymmetry(alloy_name_list, feat_key="K_GPa")

# create composition-weighted average features
df_feats["VEC_avg"] = get_comp_avg(alloy_name_list, feat_key="VEC")
df_feats["Tm_avg"] = get_comp_avg(alloy_name_list, feat_key="Tm")
df_feats["Coh_E_avg"] = get_comp_avg(alloy_name_list, feat_key="Coh_E")
df_feats["density_avg"] = get_comp_avg(alloy_name_list, feat_key="density")

df_feats.head(5)

,r_asymm,EN_Pauling_asymm,E_GPa_asymm,G_GPa_asymm,K_GPa_asymm,VEC_avg,Tm_avg,Coh_E_avg,density_avg
0,0.047618,0.076501,0.327911,0.257110,0.346523,4.2500,2328.625000,6.145000,8.10500
1,0.072823,0.073020,0.205088,0.121375,0.332279,4.6000,2237.200000,5.858000,6.32000
2,0.068739,0.076822,0.209995,0.114886,0.333395,4.5000,2250.750000,5.995000,6.40000
3,0.049710,0.081330,0.331455,0.275119,0.400897,4.4000,2523.200000,6.642000,9.86000
4,0.049601,0.082290,0.343723,0.292032,0.406759,4.3212,2433.465112,6.458601,9.45573


## NORMALIZE FEATURES

In [4]:
# Get min and max values of features
xmin = np.amin(df_feats, axis=0) #storing min of each column
xmax = np.amax(df_feats, axis=0) #storing max of each column

# Create "database_processed" directory if it does not exist 
db_processed_dir = f"database_processed{SEP}"
if not os.path.exists(db_processed_dir):
    os.makedirs(db_processed_dir)

# Save min and max values of features as csv files
xmin.to_csv(f"{db_processed_dir}xmin.csv", index=False)
xmax.to_csv(f"{db_processed_dir}xmax.csv", index=False)

# Normalize features using min-max normalization
df_feats_norm = (df_feats - xmin)/(xmax - xmin)
df_feats_norm.head(5)

,r_asymm,EN_Pauling_asymm,E_GPa_asymm,G_GPa_asymm,K_GPa_asymm,VEC_avg,Tm_avg,Coh_E_avg,density_avg
0,0.320929,0.328485,0.485293,0.372629,0.569097,0.201389,0.550645,0.645128,0.491772
1,0.504166,0.309364,0.284583,0.144037,0.541699,0.250000,0.500961,0.585305,0.327202
2,0.474475,0.330249,0.292601,0.133109,0.543845,0.236111,0.508324,0.613861,0.334578
3,0.336142,0.355013,0.491086,0.402958,0.673690,0.222222,0.656384,0.748723,0.653575
4,0.335349,0.360287,0.511133,0.431440,0.684966,0.211278,0.607619,0.710495,0.616303


## SAVE ALLOYS DATASET WITH NORMALIZED FEATURES
### This dataset will be used for training machine learning model

In [5]:
# Create a dataframe with both alloys information and feature values
df_alloys_with_norm_feats = pd.concat([df, df_feats_norm], axis=1)
df_alloys_with_norm_feats.to_excel(f"{db_processed_dir}df_alloys_with_norm_feats.xlsx", index=False)
df_alloys_with_norm_feats.head(5)

,alloy_name,phases,VHN,r_asymm,EN_Pauling_asymm,E_GPa_asymm,G_GPa_asymm,K_GPa_asymm,VEC_avg,Tm_avg,Coh_E_avg,density_avg
0,Hf0.5Nb0.5Ta0.5Ti1.5Zr,BCC,301.0,0.320929,0.328485,0.485293,0.372629,0.569097,0.201389,0.550645,0.645128,0.491772
1,NbTiV2Zr,BCC,304.0,0.504166,0.309364,0.284583,0.144037,0.541699,0.250000,0.500961,0.585305,0.327202
2,NbTiVZr,BCC,335.0,0.474475,0.330249,0.292601,0.133109,0.543845,0.236111,0.508324,0.613861,0.334578
3,HfNbTaTiZr,BCC,340.0,0.336142,0.355013,0.491086,0.402958,0.673690,0.222222,0.656384,0.748723,0.653575
4,Al0.3HfNbTaTiZr,BCC,353.0,0.335349,0.360287,0.511133,0.431440,0.684966,0.211278,0.607619,0.710495,0.616303


## READ PROCESSED DATASET WITH NORMALIZED FEATURES
### Randomize dataset using a fixed random-state

In [6]:
random_state = 0
df = pd.read_excel(f"{db_processed_dir}df_alloys_with_norm_feats.xlsx").sample(frac=1, random_state=random_state, ignore_index=True)
df.head(5)

,alloy_name,phases,VHN,r_asymm,EN_Pauling_asymm,E_GPa_asymm,G_GPa_asymm,K_GPa_asymm,VEC_avg,Tm_avg,Coh_E_avg,density_avg
0,CoFeNiSi0.75,FCC + Im,570.0,0.205132,0.000000,0.292837,0.309121,0.303021,0.722347,0.237707,0.286073,0.417694
1,HfNbSi0.5TiV,BCC + lm,490.0,0.632143,0.512060,0.151919,0.169609,0.446334,0.228333,0.519930,0.590926,0.431724
2,AlCoCrFeMo0.5Ni0.5,BCC + Im,708.0,0.398028,0.406655,0.611056,0.601850,0.538141,0.555556,0.263785,0.272642,0.410732
3,Al0.5CoCrCuFeNiTi0.6,FCC + BCC,458.0,0.399282,0.314718,0.446707,0.586140,0.373022,0.701556,0.218458,0.228859,0.436580
4,Al2.8CoCrCuFe,BCC,657.0,0.450710,0.305281,0.668399,0.889088,0.669604,0.477333,0.062948,0.153798,0.292410


## DEFINE TARGET PROPERTY AND FEATURES TO USE
### Create X and Y dataset

In [7]:
y_prop = "VHN"
feat_names = ["r_asymm", "EN_Pauling_asymm", "E_GPa_asymm", "G_GPa_asymm", "K_GPa_asymm",
              "VEC_avg", "Tm_avg", "Coh_E_avg", "density_avg"]

X = np.array(df[feat_names])
Y = np.array(df[y_prop])
print(f"X shape: {X.shape}")
print(f"Y shape: {Y.shape}")

X shape: (218, 9)
Y shape: (218,)


## DEFINE PARAMETERS TO USE FOR MODEL TRAINING

In [8]:
# REFERENCE: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
# Define parameters
Kfold_val = 5
n_estimators = 1000
criterion = "absolute_error"
max_depth = 5
min_samples_split = 2
min_samples_leaf = 1
random_state = 0

## RUN MODEL TRAINING

In [9]:
kf = KFold(n_splits=Kfold_val)

train_mae, test_mae = [], []
train_percent_err, test_percent_err = [], []
train_r2, test_r2 = [], []
    
k_count = 1

for train_indices, test_indices in kf.split(X):
    
    print(f"Running cross-validation: {k_count}", end="")
    x_train, x_test = X[train_indices], X[test_indices]
    y_train, y_test = Y[train_indices], Y[test_indices]
    
    # Create RF model
    rf_mod = RF_RGR(n_estimators=n_estimators,
                    criterion=criterion,
                    max_depth=max_depth,
                    min_samples_split=min_samples_split,
                    min_samples_leaf=min_samples_leaf,
                    random_state=random_state)
    
    # Fit RF model
    rf_mod.fit(x_train, y_train)
    
    # Calculate performance for train and test sets
    train_mae.append(round(mean_absolute_error(y_train, rf_mod.predict(x_train)), 2))
    test_mae.append(round(mean_absolute_error(y_test, rf_mod.predict(x_test)), 2))

    train_percent_err.append(round(mean_absolute_percentage_error(y_train, rf_mod.predict(x_train)), 4))
    test_percent_err.append(round(mean_absolute_percentage_error(y_test, rf_mod.predict(x_test)), 4))

    train_r2.append(round(r2_score(y_train, rf_mod.predict(x_train)), 2))
    test_r2.append(round(r2_score(y_test, rf_mod.predict(x_test)), 2))

    # Save RF model
    mod_save_dir = f"trained_models{SEP}"
    if not os.path.exists(mod_save_dir):
        os.makedirs(mod_save_dir)
    mod_savename = f"{mod_save_dir}rf-mod-K{k_count}"
    pickle.dump(rf_mod, open(mod_savename, 'wb'))
    
    k_count += 1
    print("\t...DONE.")

print("\n--- CROSS-VALIDATION PERFORMANCE ---")
print("Mean absolute error:")
print("Training: \t", train_mae)
print("Test: \t\t", test_mae)

print("\nMean absolute percentage error:")
print("Training: \t", np.array(train_percent_err)*100)
print("Test: \t\t", np.array(test_percent_err)*100)

print("\nR2 score:")
print("Training: \t", train_r2)
print("Test: \t\t", test_r2)

Running cross-validation: 1	...DONE.
Running cross-validation: 2	...DONE.
Running cross-validation: 3	...DONE.
Running cross-validation: 4	...DONE.
Running cross-validation: 5	...DONE.

--- CROSS-VALIDATION PERFORMANCE ---
Mean absolute error:
Training: 	 [38.91, 43.19, 39.47, 40.45, 40.59]
Test: 		 [71.23, 59.36, 67.51, 75.42, 71.06]

Mean absolute percentage error:
Training: 	 [10.01 11.07 10.04 10.36 10.52]
Test: 		 [18.79 16.12 20.28 22.85 14.6 ]

R2 score:
Training: 	 [0.92, 0.91, 0.92, 0.91, 0.92]
Test: 		 [0.68, 0.84, 0.72, 0.77, 0.66]
